In [12]:
import os
import random
import re

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xgboost as xgb
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor, plot_importance, plot_tree

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix, accuracy_score, r2_score
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from scipy.stats import t
import statsmodels.api as sm

## TSLW Predict

In [2]:
otu_path = r"D:\05_kuisu\project\15_MAI\Foxtail-millet-data-analysis\Figure2\OTU0.7.csv"
tslw_snp_path = r"D:\05_kuisu\project\15_MAI\Foxtail-millet-data-analysis\Figure2\TSLWSNP.csv"

df_otu = pd.read_csv(otu_path)
df_snp = pd.read_csv(tslw_snp_path)

In [3]:
tslw_idx = [0, 1007]
tslw_idx.extend([i for i in range(1, 1005)])
tslw_idx.extend([i for i in range(1014, 1024)])
df_tslw = pd.merge(df_otu.iloc[:, tslw_idx], df_snp, on="sample")
df_tslw.head()

,sample,TSLW,OTU_97,OTU_17917,OTU_164,OTU_110,OTU_28612,OTU_42,OTU_281,OTU_28306,...,PC10,snp1,snp2,snp3,snp4,snp5,snp6,snp7,snp8,snp9
0,Si_102,3.20,3.4726,1.4241,1.4241,1.9282,1.4241,2.8425,1.42410,5.1616,...,-0.039418,0.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0
1,Si_105,4.07,3.7887,1.7997,1.1639,3.2757,5.0735,3.1469,1.61720,5.2247,...,0.076554,2.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
2,Si_106,3.43,3.2836,2.1369,1.9681,2.6644,3.2836,3.4207,0.97884,5.6724,...,-0.000622,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
3,Si_108,3.23,4.1436,2.9225,2.4297,2.8365,4.9466,2.9225,0.90887,5.4595,...,0.031235,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
4,Si_109,3.53,3.3673,1.9244,1.2618,3.4310,3.5506,2.7207,0.95007,4.7600,...,0.025807,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,2.0


In [4]:
tslw_idx = [0, 1007]
tslw_idx.extend([i for i in range(1, 1005)])
tslw_idx.extend([i for i in range(1014, 1024)])
df_tslw = pd.merge(df_otu.iloc[:, tslw_idx], df_snp, on="sample")
df_tslw.head()

,sample,TSLW,OTU_97,OTU_17917,OTU_164,OTU_110,OTU_28612,OTU_42,OTU_281,OTU_28306,...,PC10,snp1,snp2,snp3,snp4,snp5,snp6,snp7,snp8,snp9
0,Si_102,3.20,3.4726,1.4241,1.4241,1.9282,1.4241,2.8425,1.42410,5.1616,...,-0.039418,0.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0
1,Si_105,4.07,3.7887,1.7997,1.1639,3.2757,5.0735,3.1469,1.61720,5.2247,...,0.076554,2.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
2,Si_106,3.43,3.2836,2.1369,1.9681,2.6644,3.2836,3.4207,0.97884,5.6724,...,-0.000622,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
3,Si_108,3.23,4.1436,2.9225,2.4297,2.8365,4.9466,2.9225,0.90887,5.4595,...,0.031235,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
4,Si_109,3.53,3.3673,1.9244,1.2618,3.4310,3.5506,2.7207,0.95007,4.7600,...,0.025807,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,2.0


In [5]:
# drop column with including NaN
data = df_tslw.dropna(axis=0, how='any')
print(data.shape)
data.head()

(680, 1025)


,sample,TSLW,OTU_97,OTU_17917,OTU_164,OTU_110,OTU_28612,OTU_42,OTU_281,OTU_28306,...,PC10,snp1,snp2,snp3,snp4,snp5,snp6,snp7,snp8,snp9
0,Si_102,3.20,3.4726,1.4241,1.4241,1.9282,1.4241,2.8425,1.42410,5.1616,...,-0.039418,0.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0
1,Si_105,4.07,3.7887,1.7997,1.1639,3.2757,5.0735,3.1469,1.61720,5.2247,...,0.076554,2.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
2,Si_106,3.43,3.2836,2.1369,1.9681,2.6644,3.2836,3.4207,0.97884,5.6724,...,-0.000622,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
3,Si_108,3.23,4.1436,2.9225,2.4297,2.8365,4.9466,2.9225,0.90887,5.4595,...,0.031235,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
5,Si_113,2.97,2.7158,1.0091,1.0091,2.4743,3.1861,3.1035,1.59710,4.9002,...,0.015700,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0


In [10]:
# Choose OTU features by P Value
results = []
for i in range(2, 1006):
    variable_name = data.columns[i]
    X = data.iloc[:, i].values.reshape(-1, 1)
    Y = data["TSLW"].values
    model = LinearRegression()
    model.fit(X, Y)
    coef = model.coef_[0]
    predictions = model.predict(X)
    n = len(X)
    mse = np.mean((Y - predictions) ** 2)
    std_err = np.sqrt(mse * (np.linalg.inv(np.dot(X.T, X)))[0, 0])
    # 计算 t 值
    t_val = coef / std_err
    # 需要样本数和自由度来计算p值
    degrees_of_freedom = n - 2
    # 计算p值 (双尾)
    p_val = 2 * (1 - t.cdf(abs(t_val), df=degrees_of_freedom))
    # 添加结果到列表中
    results.append([variable_name, coef, std_err, p_val])
r = pd.DataFrame(results, columns=["V1", "Estimate", "Std.Error", "P"])
r.head()
# Get top 200 feature by P order
r_order = r.sort_values(by=["P"], ascending=False)
r_order.head()

,V1,Estimate,Std.Error,P
239,OTU_7084,-0.000032,0.006733,0.996152
189,OTU_59,-0.000065,0.004496,0.988408
95,OTU_320,0.000173,0.010584,0.986970
427,OTU_383,0.000139,0.008467,0.986904
452,OTU_2690,-0.000175,0.009088,0.984620


In [13]:
# 获取所有列名为”snp“开头的列
cc = list(r_order["V1"][:200].values)  # 相关性较高的top200
snp_columns = [col for col in data.columns if re.match(r"^snp", col)]
otu_columns = [col for col in data.columns if re.match(r"OTU", col)]
pc_columns = [col for col in data.columns if re.match(r"^PC", col)]
print(
    f"cc nums: {len(cc)}\nsnp_columns:{len(snp_columns)}\notu_columns:{len(otu_columns)}\npc_columns:{len(pc_columns)}")
# split data to 5 part
random_data = data.sample(frac=1)
sample_nums = len(random_data)
datas = []
for i in range(5):
    sample = random_data.sample(min(sample_nums // 5, len(random_data)), replace=True)
    datas.append(sample)
    print(f"data_{i} shape: {sample.shape}")


# 使用逐步回归（向后逐步回归）
def backward_elimination(data, target, significance_level=0.05):
    initial_features = data.columns.tolist()
    while len(initial_features) > 0:
        X = sm.add_constant(data[initial_features])
        model = sm.OLS(target, X).fit()
        p_values = model.pvalues.iloc[1:]  # 不包括常数项的 p 值
        max_p_value = p_values.max()
        if max_p_value > significance_level:
            excluded_feature = p_values.idxmax()
            print(f"remove feature: {excluded_feature}, p value: {max_p_value}")
            initial_features.remove(excluded_feature)
        else:
            break
    return initial_features


cc nums: 200
snp_columns:9
otu_columns:1004
pc_columns:10
data_0 shape: (136, 1025)
data_1 shape: (136, 1025)
data_2 shape: (136, 1025)
data_3 shape: (136, 1025)
data_4 shape: (136, 1025)


In [14]:
# select SNP features
X = data[snp_columns+cc]
X.head()

,snp1,snp2,snp3,snp4,snp5,snp6,snp7,snp8,snp9,OTU_7084,...,OTU_617,OTU_14661,OTU_25514,OTU_23468,OTU_24877,OTU_5415,OTU_14036,OTU_3690,OTU_5328,OTU_20590
0,0.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0,1.6981,...,1.6981,0.00000,1.9282,1.4241,1.08570,0.00000,1.69810,2.1266,2.3011,2.1266
1,2.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,1.9616,...,1.4083,0.86958,4.9820,2.7634,2.36050,0.00000,1.16390,3.6048,2.6727,2.3605
2,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.2965,...,1.7770,0.57089,1.7770,1.5567,0.57089,0.00000,0.57089,1.7770,2.6644,2.1369
3,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,2.7451,...,3.4132,0.90887,3.4132,1.6753,0.52485,0.52485,1.46210,2.1732,1.8611,3.1535
5,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,1.5971,...,3.4759,1.33280,2.3365,1.3328,1.00910,0.00000,1.59710,1.5971,2.9226,1.8203


In [15]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 680 entries, 0 to 826
Columns: 209 entries, snp1 to OTU_20590
dtypes: float64(209)
memory usage: 1.1 MB


In [16]:
X.describe()

,snp1,snp2,snp3,snp4,snp5,snp6,snp7,snp8,snp9,OTU_7084,...,OTU_617,OTU_14661,OTU_25514,OTU_23468,OTU_24877,OTU_5415,OTU_14036,OTU_3690,OTU_5328,OTU_20590
count,680.000000,680.000000,680.000000,680.000000,680.000000,680.000000,680.000000,680.000000,680.000000,680.000000,...,680.000000,680.000000,680.000000,680.000000,680.000000,680.000000,680.000000,680.000000,680.000000,680.000000
mean,0.025000,0.936765,0.238235,0.036765,0.036765,1.060294,0.179412,0.097059,0.729412,1.697751,...,2.367972,1.038785,2.528235,2.414850,0.891638,1.261970,1.258558,2.734521,1.819393,2.005000
std,0.212201,0.990590,0.634555,0.254795,0.254795,0.989286,0.529139,0.416158,0.954188,0.868514,...,1.510253,0.858086,1.199266,0.982309,0.730016,1.061883,0.893056,0.963670,0.968746,1.010661
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.168775,...,1.293150,0.000000,1.807225,1.807400,0.000000,0.000000,0.602498,2.197450,1.194925,1.355500
50%,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,1.760200,...,2.217200,1.027450,2.482450,2.450850,0.879425,1.186200,1.216400,2.816950,1.912400,2.005900
75%,0.000000,2.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,2.000000,2.297975,...,3.414150,1.595025,3.192400,3.042300,1.409675,2.116525,1.868375,3.388075,2.474225,2.662500
max,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,4.370000,...,7.497500,4.194100,7.203800,5.894300,3.265200,4.364100,4.703000,4.973500,6.996400,4.907800


In [18]:
X.isnull().sum().sum()

0

In [19]:
## 划分测试-训练集
x_train, x_test, y_train, y_test = train_test_split(
        np.asarray(X),
        np.asarray(Y),
        train_size=0.8,
        random_state=33
    )
print(f"train:{x_train.shape}, test: {x_test.shape}")

train:(544, 209), test: (136, 209)


In [23]:
# xgboost训练
from sklearn.ensemble import GradientBoostingRegressor

# model=xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468,
#                              learning_rate=0.05, max_depth=3,
#                              min_child_weight=1.7817, n_estimators=2200,
#                              reg_alpha=0.4640, reg_lambda=0.8571,
#                              subsample=0.5213,
#                              random_state =7, nthread = -1)
model = XGBRegressor(
            learning_rate=0.01,
            n_estimators=3000,  # 树的个数--100棵树建立xgboost
            max_depth=12,  # 树的深度
            min_child_weight=2,  # 叶子节点最小权重
            gamma=0.4,  # 惩罚项中叶子结点个数前的参数
            subsample=0.7,  # 随机选择70%样本建立决策树
            colsample_bytree=0.7,  # 随机选择70%特征建立决策树
            objective='reg:squarederror',  # 使用平方误差作为损失函数
            random_state=1,  # 随机数
            reg_alpha=2,
            reg_lambda=2,
        )

model.fit(
            x_train,
            y_train,
            eval_set=[(x_test, y_test), (x_train, y_train)],
            # early_stopping_rounds=10,
            # eval_metric=["rmse"],
            # verbose=True
        )
pred=model.predict(x_test)
print('mse:',mean_squared_error(pred,y_test))
print('rmse:',np.sqrt(mean_squared_error(pred,y_test)))
print('mae:',mean_absolute_error(pred,y_test))
print('r2:',r2_score(pred,y_test))

[0]	validation_0-rmse:0.32198	validation_1-rmse:0.33738
[1]	validation_0-rmse:0.32191	validation_1-rmse:0.33684
[2]	validation_0-rmse:0.32199	validation_1-rmse:0.33612
[3]	validation_0-rmse:0.32182	validation_1-rmse:0.33553
[4]	validation_0-rmse:0.32170	validation_1-rmse:0.33506
[5]	validation_0-rmse:0.32157	validation_1-rmse:0.33456
[6]	validation_0-rmse:0.32159	validation_1-rmse:0.33395
[7]	validation_0-rmse:0.32142	validation_1-rmse:0.33354
[8]	validation_0-rmse:0.32129	validation_1-rmse:0.33292
[9]	validation_0-rmse:0.32145	validation_1-rmse:0.33237
[10]	validation_0-rmse:0.32136	validation_1-rmse:0.33179
[11]	validation_0-rmse:0.32127	validation_1-rmse:0.33139
[12]	validation_0-rmse:0.32115	validation_1-rmse:0.33093
[13]	validation_0-rmse:0.32083	validation_1-rmse:0.33022
[14]	validation_0-rmse:0.32068	validation_1-rmse:0.32967
[15]	validation_0-rmse:0.32046	validation_1-rmse:0.32909
[16]	validation_0-rmse:0.32038	validation_1-rmse:0.32850
[17]	validation_0-rmse:0.32013	validation

In [10]:
for p, label in zip(pred, y_test):
    print("predict: {:.2}, label: {:.2}".format(p, label))

predict: 2.5, label: 2.6
predict: 2.7, label: 2.8
predict: 2.7, label: 2.2
predict: 2.9, label: 2.8
predict: 2.7, label: 2.5
predict: 2.6, label: 2.4
predict: 2.7, label: 2.8
predict: 2.6, label: 3.0
predict: 2.8, label: 2.6
predict: 2.6, label: 2.5
predict: 2.5, label: 2.7
predict: 2.4, label: 2.3
predict: 2.7, label: 2.4
predict: 2.8, label: 3.1
predict: 2.6, label: 2.8
predict: 2.7, label: 2.6
predict: 2.6, label: 2.5
predict: 2.5, label: 2.6
predict: 2.7, label: 3.2
predict: 2.5, label: 2.3
predict: 2.7, label: 2.9
predict: 2.9, label: 2.7
predict: 2.8, label: 2.5
predict: 2.6, label: 2.7
predict: 2.6, label: 3.6
predict: 2.8, label: 2.5
predict: 2.6, label: 2.8
predict: 2.5, label: 2.5
predict: 2.6, label: 3.3
predict: 2.8, label: 3.0
predict: 2.9, label: 3.1
predict: 2.8, label: 2.7
predict: 2.6, label: 2.6
predict: 2.7, label: 2.5
predict: 2.6, label: 2.6
predict: 2.6, label: 2.9
predict: 2.7, label: 2.5
predict: 2.8, label: 3.0
predict: 2.6, label: 2.4
predict: 2.8, label: 2.2


In [63]:
# xgboost训练
from sklearn.ensemble import GradientBoostingRegressor

model=xgb.XGBRegressor(
    reg_alpha=3,
    reg_lambda=5,
    objective="reg:squarederror",
    booster="gbtree"
)
model.fit(x_train,y_train)
pred=model.predict(x_test)
print('mse:',mean_squared_error(pred,y_test))
print('rmse:',np.sqrt(mean_squared_error(pred,y_test)))
print('mae:',mean_absolute_error(pred,y_test))
print('r2:',r2_score(pred,y_test))

mse: 0.12083030588050621
rmse: 0.3476065388920442
mae: 0.27408550819718697
r2: -6.083289110201569


In [62]:
for p, label in zip(pred, y_test):
    print("{:.4}, {:.4}".format(p, label))

2.324, 2.57
3.072, 2.83
2.3, 2.23
2.084, 2.8
2.871, 2.53
2.339, 2.4
2.389, 2.8
2.603, 3.03
2.362, 2.6
2.472, 2.5
3.014, 2.7
2.935, 2.27
2.656, 2.37
2.828, 3.1
2.626, 2.83
2.467, 2.6
2.481, 2.53
2.727, 2.6
2.723, 3.23
2.756, 2.27
2.577, 2.94
2.601, 2.73
2.894, 2.53
2.551, 2.7
2.955, 3.6
2.579, 2.47
2.769, 2.83
2.45, 2.53
2.277, 3.3
2.933, 3.0
2.896, 3.13
2.525, 2.7
2.927, 2.57
3.088, 2.47
2.947, 2.6
2.915, 2.87
2.838, 2.5
2.638, 2.97
2.664, 2.4
2.901, 2.2
2.644, 2.5
2.737, 2.83
3.068, 2.57
2.988, 3.0
2.621, 2.73
2.832, 2.43
2.442, 2.73
2.73, 3.3
2.555, 2.63
2.508, 3.07
2.494, 2.73
2.763, 3.0
2.436, 2.67
2.711, 3.1
2.804, 2.33
2.862, 3.53
2.632, 1.93
2.801, 2.73
2.441, 2.8
2.624, 2.95
2.589, 3.47
2.491, 2.97
3.024, 3.33
2.483, 2.4
2.664, 2.87
2.09, 2.27
2.851, 2.97
2.494, 3.4
2.491, 2.87
2.882, 1.97
2.899, 2.57
2.609, 3.07
2.53, 2.53
2.59, 2.6
2.279, 2.5
2.256, 2.87
2.637, 2.33
2.496, 2.33
2.499, 1.93
2.257, 2.97
3.037, 2.63
2.675, 2.7
2.547, 2.37
2.825, 2.43
2.422, 2.5
2.838, 2.17
2.63,

In [64]:
# 线性回归
from sklearn.linear_model import LinearRegression, ElasticNet

model=LinearRegression()
model.fit(x_train,y_train)
pred=model.predict(x_test)
print('mse:',mean_squared_error(pred,y_test))
print('rmse:',np.sqrt(mean_squared_error(pred,y_test)))
print('mae:',mean_absolute_error(pred,y_test))
print('r2:',r2_score(pred,y_test))

mse: 0.3533087949285985
rmse: 0.5943978422980677
mae: 0.46531510834741896
r2: -0.08256896052273666


In [66]:
x_train.shape

(661, 1004)

In [20]:
for p, label in zip(pred, y_train):
    print("{:.4}, {:.4}".format(p, label))

2.27, 2.27
3.17, 3.17
2.17, 2.17
1.97, 1.97
2.93, 2.93
2.33, 2.33
2.27, 2.27
2.57, 2.57
2.33, 2.33
2.43, 2.43
3.03, 3.03
2.9, 2.9
2.6, 2.6
2.8, 2.8
2.67, 2.67
2.47, 2.47
2.47, 2.47
2.73, 2.73
2.7, 2.7
2.8, 2.8
2.57, 2.57
2.67, 2.67
2.9, 2.9
2.53, 2.53
3.03, 3.03
2.53, 2.53
2.8, 2.8
2.47, 2.47
2.17, 2.17
2.93, 2.93
2.9, 2.9
2.5, 2.5
3.0, 3.0
3.17, 3.17
3.0, 3.0
2.9, 2.9
2.83, 2.83
2.63, 2.63
2.63, 2.63
2.9, 2.9
2.67, 2.67
2.67, 2.67
3.13, 3.13
3.07, 3.07
2.67, 2.67
2.93, 2.93
2.25, 2.25
2.77, 2.77
2.5, 2.5
2.43, 2.43
2.47, 2.47
2.77, 2.77
2.4, 2.4
2.73, 2.73
2.73, 2.73
2.87, 2.87
2.6, 2.6
2.83, 2.83
2.33, 2.33
2.57, 2.57
2.57, 2.57
2.5, 2.5
3.13, 3.13
2.43, 2.43
2.67, 2.67
1.87, 1.87
2.9, 2.9
2.5, 2.5
2.4, 2.4
3.0, 3.0
2.93, 2.93
2.6, 2.6
2.47, 2.47
2.6, 2.6
2.17, 2.17
2.23, 2.23
2.57, 2.57
2.5, 2.5
2.47, 2.47
2.1, 2.1
3.03, 3.03
2.63, 2.63
2.57, 2.57
2.83, 2.83
2.43, 2.43
2.87, 2.87
2.63, 2.63
3.1, 3.1
2.63, 2.63
2.73, 2.73
2.37, 2.37
2.93, 2.93
2.97, 2.97
2.73, 2.73
2.03, 2.03
2.57, 2